# What "types" of cell towers do we have?

In this notebook we will consider the question of if we can find different 'types' of cell towers based on usage activity.

As part of this we will have a few things to figure out:

* What feature representation will we use for each cell?
* What clustering approach will we use?
* How will we then try to make sense and interpret the clusters?

We will touch on each of these considerations as we go. And like any DS project there are countless ways we could go about formulating the problem, below is an initial approach which no doubt could be iterated and improved on. 

## Setup

Imports and some settings...

In [ ]:
import pandas as pd
import numpy as np
import glob
import hdbscan
from sklearn.cluster import KMeans
import seaborn as sns
import matplotlib.pyplot as plt
import geojson
import matplotlib.colors as colors
import matplotlib.cm as cmx
from descartes import PolygonPatch

%matplotlib inline

# set path to where data and files are
MY_DIR = r'C:\Users\Andrew\github\mobile-phone-activity'

## Get data

Pull in each csv file one by one and then append them all into __df_big__ and a smaller version of the full data __df_small__ (for experimentation or quick development).

In [ ]:
# get data

# get file paths
path = MY_DIR
files = glob.glob(path + "\data\sms-call-internet-mi-2013-11-*")

# empty list to store individual df's into 
list_ = []

# loop through files
for file_ in files:
    df_tmp = pd.read_csv(file_,index_col=None, header=0)
    list_.append(df_tmp)

# concat all the df's into one
df_big = pd.concat(list_)

# make a small version of the full data for developing with
df_small = df_big.sample(n = 10000)

## Data prep

First we will just rename some columns and print out some summary stats and info on our data...

In [ ]:
# pick big or small df to work with
#df = df_small.copy()
df = df_big.copy()

# clean up some col names
df.rename(columns={'CellID':'cell_id'}, inplace=True)

# ensure cell_id is a string (for count uniques in describe())
df['cell_id'] = df['cell_id'].astype(str)

# ensure datetime is correct format
df['datetime'] = pd.to_datetime(df['datetime'])

print(df.shape)
df.sample(5)

Print out some info on the data...

In [ ]:
# look at some info about df
df.info(verbose=True)

Print out summary stats for each column...

In [ ]:
df.describe(include='all')

### Missing data

In this case we are safe to set missing data values to be 0.

So we are assuming the absence of measurements for any given cell in a specific hour represents a lack of traffic as opposed to any issues with the data... 

In [ ]:
# get missing value percentages
print("% Missing Values")
print(df.isnull().sum()/len(df)*100)

...looks from above that there is indeed quite a bit of missing data here. Given the data is already aggregated up to cell and hour level i'd probably be asking questions on this a bit more. Is it really common enoungh for a cell to go a whole hour without any call's in or out?<br><br>Not much we can do about that here so we will take it at face value...

In [ ]:
# fill all nulls with 0
df.fillna(value=0, inplace=True)

In [ ]:
# look at a sample of data and keep track of the df shape as we do stuff
print(df.shape)
df.sample(5)

## Design considerations

We will build some features to help us aggregate up from day and hour to "weekday" or "weekend" and "time of day" (night, day, or evening). 

This will help us form a representation that should capture certain way's we think the data might behave while also being as compact as possible so as to avoid having lots of features feeding into the clustering which would be subject to the [curse of dimensionality](https://en.wikipedia.org/wiki/Curse_of_dimensionality#Distance_functions).

This is one point where we are making a specific design decision, as always its one we could play around with and revisit as we iterate (e.g. why not daily level or slice the day into different time bins).

In [ ]:
# get if weekend or weekday from datetime
df['weekday'] = np.where((df['datetime'].dt.dayofweek < 5) , 'weekday', 'weekend')

# segment hours into times of day
def timeofday(hour):
    if hour <= 6:
        return 'night'
    elif hour <= 17:
        return 'day'
    elif hour <= 23:
        return 'evening'
df['timeofday'] = df['datetime'].dt.hour.apply(timeofday)

# make a single key field to describe day and time of traffic data
df['day_time'] = df['weekday'] + '_' + df['timeofday']

# drop some columns we no longer need
del df['datetime']
del df['countrycode']
del df['weekday']
del df['timeofday']

print(df.shape)
df.sample(5)

## Metrics as percentages

We will transform our data to be percentages - this idea here is to help with the interpretation and it also acts as a sort of way to normalize all our data to be on a similar scale which is key for distance measures when clustering.

Again, this is a design choice, assuming we are really interested in the bahaviour of the traffic in terms of its eb's and flows then it's reasonable to focus on percentages. However one trade off here is that any notion of the size of a cell in terms of the volume of traffic it handles is no longer possible.  

So we will first get the total values for each metric across the whole week and then we will use that to transform into percentages - so for example "callsin" will no become the % of all traffic in the sample period for that specific cell that falls on that specific "day_time" slice. 

In [ ]:
# get cell totals for the whole week
df_cell_totals = df.groupby(['cell_id'], as_index=False).sum()

# get overall volumes regardless of direction
df_cell_totals['calltotal'] = abs(df_cell_totals['callin']) + abs(df_cell_totals['callout'])
df_cell_totals['smstotal'] = abs(df_cell_totals['smsin']) + abs(df_cell_totals['smsout'])
df_cell_totals['internettotal'] = df_cell_totals['internet']

print(df_cell_totals.shape)
df_cell_totals.sample(5)

Now we will aggregate up the low level data to cell and day_time level - summing all metrics as we go...

In [ ]:
# sum up to day_time, cell level
df = df.groupby(['day_time','cell_id'], as_index=False).sum()

print(df.shape)
df.sample(5)

Now we will create __df_pct__ dataframe where the metrics have all been transformed into percentages of cell total traffic for each metric.

In [ ]:
# merge in cell totals
df_pct = pd.merge(df,df_cell_totals[['cell_id','internettotal','calltotal','smstotal']],how='left', on='cell_id')

# fill all nulls with 0
df_pct.fillna(value=0, inplace=True)

# convert measures to percentages of totals for that cell for the sample
df_pct['smsin'] = df_pct['smsin']/df_pct['smstotal']
df_pct['smsout'] = df_pct['smsout']/df_pct['smstotal']
df_pct['callin'] = df_pct['callin']/df_pct['calltotal']
df_pct['callout'] = df_pct['callout']/df_pct['calltotal']
df_pct['internet'] = df_pct['internet']/df_pct['internettotal']

# fill all nulls with 0
df_pct.fillna(value=0, inplace=True)

print(df_pct.shape)
df_pct.sample(5)

In [ ]:
# summary stats
df_pct.describe(include='all')

## Long to wide

As we are interested in clustering cell towers we need to transform our data to be one row per cell. This is typically called moving from a long format to a wide format whereby we will pivot our data so make more columns for each type of row relating to a cell.

So really we want to make our data wider with groups of metrics for each day_time value for each cell...

In [ ]:
# now pivot to go wide with metrics for each day_time and one row per cell
df_wide = df_pct[['cell_id',
                  'day_time',
                  'smsin',
                  'smsout',
                  'callin',
                  'callout',
                  'internet']].pivot(index='cell_id', columns='day_time')

# fill in nulls
df_wide.fillna(0, inplace=True)

print(df_wide.shape)
df_wide.sample(5)

...We now have a hierarchial column index after our pivot - to make things easier we will flatten this and rename columns to be more meaningful...

In [ ]:
# get new column names so we can flatten the nested index
new_col_names = []

# loop through each level and build up new col names
for c1 in df_wide.columns.levels[0]:
    for c2 in df_wide.columns.levels[1]:
        new_col_name = c1 + '_' + c2
        new_col_names.append(new_col_name)

#print(new_col_names)
df_wide.columns = df_wide.columns.droplevel()
df_wide.columns = new_col_names

print(df_wide.shape)
df_wide.sample(5)

...note: we now have 10,000 rows of data as hoped - one per cell... 

In [ ]:
df_wide.info(verbose=True)

In [ ]:
df_wide.describe(include='all')

## Do Clustering

Now we are ready to perform our clustering and explore the results...

In [ ]:
# make a new df to use from here on
df_final = df_wide.copy()

print(df_final.shape)
df_final.sample(5)

Here we will just start simple with a k-means approach. We have iterated on a few different choices of k and settled on a final value. 

We could use someting a bit more fancy like [HDBSCAN](http://hdbscan.readthedocs.io/en/latest/) which can fit more flexible cluster's and have a bit more of a data driven approach to the number of clusters. It also could be useful for helping pick out outlier cells - those for which no cluster was found to be close enough. 

But to keep it simple for now we will stick with k-means...

In [ ]:
# define clustering approach
#clusterer = hdbscan.HDBSCAN()
clusterer = KMeans(n_clusters=10)

In [ ]:
# do the clustering
clusterer.fit(df_final)
print(clusterer.labels_.max())

In [ ]:
# add the cluster labels back onto the input data
df_final['cluster'] = clusterer.labels_
#df_final['cluster_prob'] = clusterer.probabilities_ # onlt relvant for HDBSCAN

In [ ]:
print(df_final.shape)
df_final.sample(5)

Lets look at the amount of cells in each cluster to get a feel for how cells have been distributed to each cluster...

In [ ]:
# looks at % in each cluster
cluster_counts = df_final['cluster'].value_counts()
cluster_percents = cluster_counts / len(df_final['cluster'])
print(cluster_counts)
print(cluster_percents)

## Explore Clustering

Now we will do some plots to help us explore and understand each cluster as well as get some feel for what the main differences between the clusters are...

In [ ]:
# for each input metric plot a boxplot and violin plot to get a feel for how distributions differ by cluster
for var in df_final.columns[0:30]:
    fig, (ax1, ax2) = plt.subplots(ncols=2, sharey=True, figsize=(12,6))
    ax1.set_title(var)
    ax2.set_title(var)
    sns.boxplot(x="cluster", y=var, data=df_final, ax=ax1)
    sns.violinplot(x="cluster", y=var, data=df_final, ax=ax2)

...from the above boxplots we can get a feel for the variables that differ a lot accors clusters and those that don't...

In [ ]:
# for each input metric plot a kernel density plot to get a feel for how distributions differ by cluster
for var in df_final.columns[0:30]:
    fig, ax = plt.subplots(figsize=(12,6))
    labels = []
    for key, grp in df_final.groupby(['cluster']):
        ax = grp.plot(ax=ax, kind='kde', x='cluster', y=var)
        labels.append(key)
    lines, _ = ax.get_legend_handles_labels()
    ax.legend(lines, labels, loc='best')
    ax.set_title(var)
    plt.show()

...similar to the box plots the density plots help us get a feel for the variables on which the clusters differ the most or least.

Next we will try and take a more hollistic view and make some point plots for each cluster for each metric. This is a nice quick way to see where some of the main differences in each cluster are. 

To make this plot work however we have to summarise each metric in each cluster. So we will start with the median values of each metric per cluster and then also look at metrics like the mean and different quantile thresholds (to get a feel for the spread).

In [ ]:
# get a long df giving the median value for each metric for each cluster
df_long = pd.melt(df_final.groupby('cluster', as_index=False).median(), id_vars=['cluster'])
agg_mode = 'Median'
title = 'Median Values of Input Features By Cluster'
plt.figure(figsize=(20,5))
ax = sns.pointplot(x="variable", y="value", hue="cluster", data=df_long, markers='o', linestyles='-')
ax.set_ylabel(agg_mode)
ax.set_title(title)
plt.xticks(rotation=90)

...so for example, from the above plot we can see that the main metric the clusters differ on is "internet_weekday_day" ranging from around 30% to 60%.

In [ ]:
# same as above but look at averages
df_long = pd.melt(df_final.groupby('cluster', as_index=False).mean(), id_vars=['cluster'])
agg_mode = 'Mean'
title = 'Mean Values of Input Features By Cluster'
plt.figure(figsize=(20,5))
ax = sns.pointplot(x="variable", y="value", hue="cluster", data=df_long, markers='o', linestyles='-')
ax.set_ylabel(agg_mode)
ax.set_title(title)
plt.xticks(rotation=90)

In [ ]:
# 25th percentile
df_long = pd.melt(df_final.groupby('cluster', as_index=False).quantile(.25), id_vars=['cluster'])
df_long.columns = ['cluster','variable','value']
agg_mode = '25th Percentile'
title = '25th Percentile Values of Input Features By Cluster'
plt.figure(figsize=(20,5))
ax = sns.pointplot(x="variable", y="value", hue="cluster", data=df_long, markers='o', linestyles='-')
ax.set_ylabel(agg_mode)
ax.set_title(title)
plt.xticks(rotation=90)

In [ ]:
# 75th percentile
df_long = pd.melt(df_final.groupby('cluster', as_index=False).quantile(.75), id_vars=['cluster'])
df_long.columns = ['cluster','variable','value']
agg_mode = '75th Percentile'
title = '75th Percentile Values of Input Features By Cluster'
plt.figure(figsize=(20,5))
ax = sns.pointplot(x="variable", y="value", hue="cluster", data=df_long, markers='o', linestyles='-')
ax.set_ylabel(agg_mode)
ax.set_title(title)
plt.xticks(rotation=90)

### Look at summary stats for each cluster visually

Next lets look at each cluster on its own and plot similar metrics but just for one cluster at a time so we can get a feel for the range of values within each cluster...

In [ ]:
# make lots of long summary stat tables for each metric and cluster
df_long_10 = pd.melt(df_final.groupby('cluster', as_index=False).quantile(.1), id_vars=['cluster'])
df_long_10.columns = ['cluster','variable','value']
df_long_10['metric'] = '10th Percentile'
df_long_25 = pd.melt(df_final.groupby('cluster', as_index=False).quantile(.25), id_vars=['cluster'])
df_long_25.columns = ['cluster','variable','value']
df_long_25['metric'] = '25th Percentile'
df_long_50 = pd.melt(df_final.groupby('cluster', as_index=False).quantile(.50), id_vars=['cluster'])
df_long_50.columns = ['cluster','variable','value']
df_long_50['metric'] = '50th Percentile'
df_long_75 = pd.melt(df_final.groupby('cluster', as_index=False).quantile(.75), id_vars=['cluster'])
df_long_75.columns = ['cluster','variable','value']
df_long_75['metric'] = '75th Percentile'
df_long_90 = pd.melt(df_final.groupby('cluster', as_index=False).quantile(.75), id_vars=['cluster'])
df_long_90.columns = ['cluster','variable','value']
df_long_90['metric'] = '90th Percentile'

In [ ]:
# union all the stats data together
df_long_stats = df_long_10.append([df_long_25,df_long_50,df_long_75,df_long_90])

print(df_long_stats.shape)
df_long_stats.sample(5)

In [ ]:
# now loop through each cluster and plot the summary stats for each metric
for c in clusterer.labels_: 
    df_long = df_long_stats[(df_long_stats.cluster == c)]
    agg_mode = 'Summary Metric Value'
    title = 'Summary Stats For Cluster ' + str(c)
    plt.figure(figsize=(20,5))
    ax = sns.pointplot(x="variable", y="value", hue="metric", data=df_long, markers='o', linestyles='-')
    ax.set_ylabel(agg_mode)
    ax.set_title(title)
    plt.xticks(rotation=90)
    plt.show()

...from the above plots we can get a sense for the variance within each metric for each cluster by seeing the spread of the different percentile lines. 

## Plot Clusters On Map

Finally we will plot the cluster on the actual cell grid map to see if any interesting geo spatial patterns jump out. 

In [ ]:
# load in the geo json data
with open(MY_DIR + "\\milano-grid.geojson") as json_file:
    json_data = geojson.load(json_file)

In [ ]:
fig = plt.figure() 
ax = fig.gca() 

# some things to handle the coloring
jet = cm = plt.get_cmap('Paired') 
cNorm  = colors.Normalize(vmin=0, vmax=clusterer.labels_.max())
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)

# go through each cell and color it by cluster
for i in range(1,len(json_data.features)):
    
    # put a try block in in case can't find some cells in the cluster data for some reason.
    
    try:
        poly = json_data.features[i]['geometry']
        cell_id = str(json_data.features[i]['properties']['cellId'])
        cluster = df_final.loc[cell_id]['cluster']
        colorVal = scalarMap.to_rgba(cluster)
        
        # add patch to the map
        ax.add_patch(PolygonPatch(poly, fc=colorVal, ec=colorVal, alpha=0.8, zorder=1 ))
        
    except:
        pass

ax.axis('scaled')

fig.set_size_inches(11,11)
plt.title("Cell Map Of Cluster Membership")
plt.show()

The most interesting thing about this map is that there does indeed seem to be spatial patterns here with nearby cells tending to be in the same cluster and 'blobs' of clusters seeming to have formed. This is nice to see as if this were not the case then that would raise suspisions on the clustering approach as having done anything much different to a random assignment.

To illustrate this point a bit we will do the same chart as above but with a random clusrter assignemnt to illustrate the difference...

In [ ]:
fig = plt.figure() 
ax = fig.gca() 

# some things to handle the coloring
jet = cm = plt.get_cmap('Paired') 
cNorm  = colors.Normalize(vmin=0, vmax=clusterer.labels_.max())
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)

# go through each cell and color it by cluster
for i in range(1,len(json_data.features)):
    
    # put a try block in in case can't find some cells in the cluster data for some reason.
    
    try:
        poly = json_data.features[i]['geometry']
        cell_id = str(json_data.features[i]['properties']['cellId'])
        #cluster = df_final.loc[cell_id]['cluster']
        cluster = np.random.choice(clusterer.labels_,1)[0] # randomly pick a cluster
        colorVal = scalarMap.to_rgba(cluster)
        
        # add patch to the map
        ax.add_patch(PolygonPatch(poly, fc=colorVal, ec=colorVal, alpha=0.8, zorder=1 ))
        
    except:
        pass

ax.axis('scaled')

fig.set_size_inches(11,11)
plt.title("Cell Map Of Cluster Membership")
plt.show()